### **Install and Imports**

In [ ]:
# !pip install numpy emoji langdetect contractions beautifulsoup4 regex


In [16]:
#Import pandas library for data processing data file
import pandas as pd
from bs4 import BeautifulSoup
import csv
import os
import numpy as np
import emoji
import textwrap # textwrap library for formatting text
from langdetect import detect, DetectorFactory
from collections import Counter
import regex as re
import unicodedata
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
import contractions 
from langdetect import detect, detect_langs, DetectorFactory
import pandas as pd
from collections import Counter
from emoji import EMOJI_DATA
from nltk.corpus import stopwords, wordnet as wn
from nltk.stem import WordNetLemmatizer


In [2]:
pd.set_option("display.max_colwidth", None)  # show full column content

### **Read from CSV file**


In [17]:
# df = pd.read_csv('2020_LD_balanced.csv')
# df = df[['uid', 'label', 'tweet']]
df = pd.read_csv('2024_LD_balanced.csv')
df = df[['uid', 'label', 'tweet']]

In [19]:
#rename cloumn text to tweet 
df = df.rename(columns={'text': 'tweet'})

**Extract and remove hashtags**

In [20]:
# save hashtage for each tweet
df['hashtags'] = df['tweet'].apply(lambda x: re.findall(r"#\w+", x))
# just drop the # symbol but keep the text
# df['cleaned_tweet'] = df['cleaned_tweet'].apply(lambda x: re.sub(r"#\w+", "", x).strip())
# just drop the # symbol but keep the text
df['tweet'] = df['tweet'].apply(lambda x: re.sub(r"#(\w+)", r"\1", str(x)).strip())

In [21]:
df.head(5)

,uid,label,tweet,hashtags
0,278800,Left,Should the IRS let Americans deduct car loan i...,[]
1,349127,Left,@DOGGO525 @WhiskeyMD247365 @SweDiscoLovers @NB...,[]
2,20481,Left,For sure it would be!! Glad to see some sanity...,[]
3,86511,Left,DEMOCRATS BELIEVE THEY ARE ABOVE THE LAW !! We...,[]
4,14682,Left,They don't have a clue about economics and how...,"[#AmericanWay, #TRUMP2024, #Unity]"


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9606 entries, 0 to 9605
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   uid       9606 non-null   int64 
 1   label     9606 non-null   object
 2   tweet     9606 non-null   object
 3   hashtags  9606 non-null   object
dtypes: int64(1), object(3)
memory usage: 300.3+ KB


**Remove URL and USER Names**

In [23]:
# Regex patterns
urlPattern = re.compile(r'https?://\S+')
mentionPattern = re.compile(r'@\w+')
# function to replaces @usernames with @user and URLs with 'http'.
def maskUsernamesURL(text: str) -> str:
    text = mentionPattern.sub("@user", str(text))
    text = urlPattern.sub("http", text)
    return text


In [24]:
df["tweet"] = df["tweet"].apply(maskUsernamesURL)
df.head()

,uid,label,tweet,hashtags
0,278800,Left,Should the IRS let Americans deduct car loan i...,[]
1,349127,Left,@user @user @user @user You mean like when Ips...,[]
2,20481,Left,For sure it would be!! Glad to see some sanity...,[]
3,86511,Left,DEMOCRATS BELIEVE THEY ARE ABOVE THE LAW !! We...,[]
4,14682,Left,They don't have a clue about economics and how...,"[#AmericanWay, #TRUMP2024, #Unity]"


**Punctuation Removal**


In [25]:
def removePunctuation(text):
    # remove HTML
    text = BeautifulSoup(text, "html.parser").get_text()   
    # replace connectors with space    
    text = re.sub(r'[-/_]', ' ', text)                         
    # remove punctuation but keep @, #, and emojis
    text = re.sub(r'[^\p{L}\p{N}\s@#\p{Emoji}]', '', text) # (unutbu, 2014)
    # collapse multiple spaces
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [26]:
df['tweet'] = df['tweet'].apply(removePunctuation)
df.head()

,uid,label,tweet,hashtags
0,278800,Left,Should the IRS let Americans deduct car loan i...,[]
1,349127,Left,@user @user @user @user You mean like when Ips...,[]
2,20481,Left,For sure it would be Glad to see some sanity f...,[]
3,86511,Left,DEMOCRATS BELIEVE THEY ARE ABOVE THE LAW We Mu...,[]
4,14682,Left,They dont have a clue about economics and how ...,"[#AmericanWay, #TRUMP2024, #Unity]"


In [27]:
def removeUnicodeScript(text): 
    if not isinstance(text, str):
        text = str(text)
    # remove control characters (C = Control)
    text = "".join(ch for ch in text if unicodedata.category(ch)[0] != "C") # (Quinn, 2013)
    # remove directional formatting marks
    text = re.sub(r'[\u2066-\u2069]', "", text) # (Quinn, 2013)
    # keep only Latin letters, digits, punctuation, and whitespace
    text = re.sub(r'[^\p{Latin}\d\s\p{P}\p{Emoji}]', '', text)  # (unutbu, 2014)
    return text

In [28]:
df['tweet'] = df['tweet'].apply(removeUnicodeScript)
df.head()

,uid,label,tweet,hashtags
0,278800,Left,Should the IRS let Americans deduct car loan i...,[]
1,349127,Left,@user @user @user @user You mean like when Ips...,[]
2,20481,Left,For sure it would be Glad to see some sanity f...,[]
3,86511,Left,DEMOCRATS BELIEVE THEY ARE ABOVE THE LAW We Mu...,[]
4,14682,Left,They dont have a clue about economics and how ...,"[#AmericanWay, #TRUMP2024, #Unity]"


In [29]:
#  tokenize tweets and store results in a new column 'tokens'.
def tokenizeTweets(df, text_col="tweet"): 
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=False, reduce_len=True)
    df["tokens"] = df[text_col].apply(lambda x: tokenizer.tokenize(str(x)))
    return df # (Lane, 2021)

In [30]:
# POS helper — maps POS tags to WordNet categories
def get_pos(tag): # (Deepak, 2017; GeeksforGeeks, 2024b)
    if tag.startswith("V"):
        return wn.VERB
    elif tag.startswith("R"):
        return wn.ADV
    elif tag.startswith("J"):
        return wn.ADJ
    else:
        return wn.NOUN

# Function to clean, lemmatize, and rejoin one text entry
def processTweets(tokens):
    # 1. Tokenize
    # tokens = word_tokenize(str(text))

    # # 2. Remove stopwords but keep ideological terms
    # stopwords_list = set(stopwords.words("english"))
    # ideological_terms = {
    #     "left","right","liberal","liberals","conservative","conservatives",
    #     "moderate","centrist","progressive","progressives","populist","populists",
    #     "socialist","socialists","capitalist","capitalists","marxist","communist",
    #     "democrat","democrats","republican","republicans","libertarian","authoritarian",
    #     "patriot","patriotic","nationalist","nationalists","globalist","globalists",
    #     "feminist","feminists","woke","antiwoke","antifa","alt","rightwing","leftwing",
    #     "religious","christian","muslim","jewish","secular","faith","family","patriotism",
    #     "abortion","gun","guns","climate","climatechange","environment","immigration",
    #     "border","security","tax","taxes","inflation","economy","healthcare","covid",
    #     "mask","vaccine","freedom","rights","justice","inequality","race","racism",
    #     "lgbt","gender","transgender","education","crime","police","defund","war",
    #     "ukraine","israel","palestine","terrorism","peace","prolife","choice",
    #     "studentdebt","minimumwage","unions","labor","wealth","elites","media"
    # }
    # stopwords_list -= ideological_terms
    # tokens = [t for t in tokens if t.lower() not in stopwords_list]

    # Lemmatize (POS-aware) # (Deepak, 2017; GeeksforGeeks, 2024b)
    wordLem = WordNetLemmatizer()
    tagged = nltk.pos_tag(tokens)
    lemmas = [wordLem.lemmatize(w, get_pos(p)) for w, p in tagged]

    # Join back into a cleaned string
    return " ".join(lemmas)


In [31]:
df = tokenizeTweets(df) # takes tweet column and creates tokens column
df["text"] = df["tokens"].apply(processTweets)
# drop tokens column
df = df.drop(columns=["tokens"])
df.head()

,uid,label,tweet,hashtags,text
0,278800,Left,Should the IRS let Americans deduct car loan i...,[],should the irs let american deduct car loan in...
1,349127,Left,@user @user @user @user You mean like when Ips...,[],@user @user @user @user you mean like when ips...
2,20481,Left,For sure it would be Glad to see some sanity f...,[],for sure it would be glad to see some sanity f...
3,86511,Left,DEMOCRATS BELIEVE THEY ARE ABOVE THE LAW We Mu...,[],democrat believe they be above the law we must...
4,14682,Left,They dont have a clue about economics and how ...,"[#AmericanWay, #TRUMP2024, #Unity]",they dont have a clue about economics and how ...


**Emoticons**

In [32]:
for col in df.select_dtypes(include=['object']).columns:
        df[col] = df[col].astype(str).apply(emoji.demojize) # (carpedm20, 2025)

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9606 entries, 0 to 9605
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   uid       9606 non-null   int64 
 1   label     9606 non-null   object
 2   tweet     9606 non-null   object
 3   hashtags  9606 non-null   object
 4   text      9606 non-null   object
dtypes: int64(1), object(4)
memory usage: 375.4+ KB


#### 2020 SAVE 


In [ ]:
# ##SAVE TO CSV
# df.to_csv(
#     "For Fine/2020_LD_cleaned.csv",
#     index=False,
#     encoding="utf-8"
# )

# df[['label', 'text']].to_csv(
#     "For Fine/2020_LD_cleaned_tweets.csv",
#     index=False
# )

#### 2024 SAVE 

In [34]:
##SAVE TO CSV
df.to_csv(
    "For Fine/2024_LD_cleaned.csv",
    index=False,
    encoding="utf-8"
)

df[['label', 'text']].to_csv(
    "For Fine/2024_LD_cleaned_tweets.csv",
    index=False
)